In [1]:
import pandas as pd
import json

In [2]:
pd.set_option("display.max_columns", None)

# json -> csv

In [58]:
name_data = ["coupang", "tving", "justwatch"]
for name in name_data:
    df = pd.read_json(f'/Users/jeonggyu/Desktop/수집한 dataset/new_data/{name}_img.json', encoding='utf-8', orient='index')
    df.to_csv(f'/Users/jeonggyu/Desktop/수집한 dataset/new_data/{name}_img.csv', encoding='utf-8')

# JustWatch 정보 확인

In [85]:
jw_df = pd.read_csv('/Users/jeonggyu/Desktop/수집한 dataset/new_data/justwatch_img.csv')

In [89]:
jw_df.columns

Index(['Unnamed: 0', 'ott', 'title', 'original_title', 'category', 'country',
       'genre', 'director', 'actors', 'released_year', 'runtime', 'summary',
       'rating', 'imgurl', 'ott.coupang', 'ott.Netflix', 'ott.tving',
       'ott.Watcha', 'ott.wavve'],
      dtype='object')

# ott 서비스별로 행 나누기

In [87]:
ott = {'Netflix', 'tving', 'coupang', 'wavve', 'Watcha'}

In [88]:
for name in ott:
    jw_df[f'ott.{name}'] = None

In [90]:
for idx in jw_df.index:
    cur = jw_df.loc[idx]
    tmp = eval(cur.ott)

    for key, val in tmp.items():
        if key in ott:
            cur[f'ott.{key}'] = str(val)
    jw_df.loc[idx] = cur

/usr/local/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [91]:
jw_df = jw_df.drop(columns=['ott'])

# Justwatch 데이터 전처리

In [135]:
jw_df.rename(columns={'Unnamed: 0' : 'id'}, inplace=True)

In [92]:
# 이건 어떻게 처리할까
len(jw_df[jw_df['director'] == '[]'])

3250

In [136]:
# runtime없는 것은 어떻게? summary없는 것은 어떻게?
jw_df.isnull().sum()

id                    0
title                 0
original_title        0
category              0
country               0
genre                 0
director              0
actors                0
released_year         0
runtime              22
summary              62
rating                0
imgurl              215
ott.coupang       17755
ott.Netflix       12732
ott.tving         17755
ott.Watcha        10624
ott.wavve          5041
dtype: int64

In [482]:
# ott서비스 필요한 것만 가져옴 (넷플릭스, 티빙, 웨이브, 왓챠, 쿠팡플레이)
need_ott = set(filter(lambda x: x.startswith('ott'), jw_df.columns)) - {'ott.Netflix', 'ott.Watcha', 'ott.wavve'}
jw_df = jw_df.drop(columns=need_ott)

In [95]:
len(jw_df[jw_df.duplicated()])

0

# tving 전처리

In [107]:
tving_df = pd.read_csv('/Users/jeonggyu/Desktop/수집한 dataset/new_data/tving_img.csv')

In [108]:
tving_df.columns

Index(['Unnamed: 0', 'ott', 'title', 'original_title', 'category', 'country',
       'genre', 'director', 'actors', 'released_year', 'runtime', 'summary',
       'rating', 'imgurl'],
      dtype='object')

In [123]:
tving_df['ott.tving'] = 1
tving_df = tving_df.drop(columns=['ott'])

In [133]:
tving_df.rename(columns = {'Unnamed: 0' : 'id'}, inplace=True)

In [134]:
tving_df.head(5)

,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.tving
0,P001504387,갯마을 차차차,NaN,TV 프로그램,[],"['드라마', '로맨스']",['유제원'],"['신민아', '김선호', '이상이', '김영옥', '조한철', '공민정', '인교...",2021,0,현실주의 치과의사 윤혜진과 만능 백수 홍반장이 짠내 사람내음 가득한 바닷마을 ‘공진...,0.0,https://image.tving.com/resize_cdn.php?u=https...,1
1,P001503248,유미의 세포들,NaN,TV 프로그램,[],"['드라마', '로맨스']",['이상엽'],"['김고은', '안보현', '이유비', '박지현']",2021,0,세포들과 함께 먹고 사랑하고 성장하는 평범한 유미(김고은 분)의 이야기를 그린 세포...,0.0,https://image.tving.com/resize_cdn.php?u=https...,1
2,P001496087,환승연애,NaN,TV 프로그램,[],"['예능', '관찰리얼리티']","['이진주', '최미래', '이희선', '김슬기', '곽정은', '이승환', '신혜...","['사이먼도미닉', '이용진', '김예원', '유라']",2021,0,다양한 이유로 이별한 커플들이 모여 지나간 사랑을 되짚고 새로운 사랑을 찾아나가는 ...,0.0,https://image.tving.com/resize_cdn.php?u=https...,1
3,P000515167,놀라운 토요일,NaN,TV 프로그램,[],"['예능', '음악예능']","['이태경', '진보미', '강은비', '강유민', '김윤아', '임다영']","['신동엽', '김동현', '문세윤', '박나래', '넉살', '태연', '한해',...",2018,0,‘좋아요’ 폭발하는 SNS 핵 공감 핫 플레이스부터 오랜 전통이 살아 숨 쉬는 노포...,0.0,https://image.tving.com/resize_cdn.php?u=https...,1
4,P001515756,Show Me The Money 10,NaN,TV 프로그램,[],"['예능', '음악예능']",[],[],2021,0,Mnet <Show Me The Money 10>2021 COMING SOON,0.0,https://image.tving.com/resize_cdn.php?u=https...,1


In [125]:
print(len(tving_df['title'].unique()))
print(len(tving_df))

5694
5753


In [120]:
tving_df.isnull().sum()

Unnamed: 0           0
ott                  0
title                0
original_title    2385
category             0
country              0
genre                0
director             0
actors               0
released_year        0
runtime              0
summary              3
rating               0
imgurl               6
ott.tving            0
dtype: int64

In [111]:
len(tving_df[tving_df.duplicated()])

0

In [112]:
tving_df = tving_df.drop_duplicates()

In [113]:
len(tving_df[tving_df.duplicated()])

0

In [114]:
tving_df.isnull().sum()

Unnamed: 0           0
ott                  0
title                0
original_title    2392
category             0
country              0
genre                0
director             0
actors               0
released_year        0
runtime              0
summary              3
rating               0
imgurl               6
dtype: int64

In [115]:
len(tving_df[tving_df.duplicated(['title', 'category', 'director'])])

99

In [116]:
tving_df = tving_df.drop_duplicates(['title', 'category', 'director'])

In [117]:
len(tving_df)

5753

# Coupang 전처리

In [126]:
coupang_df = pd.read_csv('/Users/jeonggyu/Desktop/수집한 dataset/new_data/coupang_img.csv')

In [145]:
coupang_df.columns

Index(['id', 'title', 'director', 'actors', 'summary', 'country', 'category',
       'genre', 'runtime', 'rating', 'imgurl', 'is_coopang'],
      dtype='object')

In [138]:
coupang_df = coupang_df.drop(columns=['ott', 'Unnamed: 0', 'is_netflix', 'is_tving', 'is_wavve', 'is_watcha'])

In [146]:
coupang_df.rename(columns={'is_coopang' : 'ott.coupang'}, inplace=True)

In [143]:
print(len(coupang_df['title'].unique()))
print(len(coupang_df))

1791
1791


In [147]:
coupang_df.isnull().sum()

id             0
title          0
director       0
actors         0
summary        0
country        0
category       0
genre          0
runtime        0
rating         0
imgurl         0
ott.coupang    0
dtype: int64

In [148]:
len(coupang_df[coupang_df.duplicated()])

0

In [149]:
len(coupang_df)

1791

In [150]:
len(coupang_df[coupang_df.duplicated(['title', 'category', 'director'])])

0

# tving + coupang

In [151]:
tv_cou_df = pd.concat([tving_df, coupang_df], ignore_index=True)

In [152]:
print(len(tv_cou_df))
tv_cou_df.head(5)

7544


,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.tving,ott.coupang
0,P001504387,갯마을 차차차,NaN,TV 프로그램,[],"['드라마', '로맨스']",['유제원'],"['신민아', '김선호', '이상이', '김영옥', '조한철', '공민정', '인교...",2021.0,0,현실주의 치과의사 윤혜진과 만능 백수 홍반장이 짠내 사람내음 가득한 바닷마을 ‘공진...,0.0,https://image.tving.com/resize_cdn.php?u=https...,1.0,NaN
1,P001503248,유미의 세포들,NaN,TV 프로그램,[],"['드라마', '로맨스']",['이상엽'],"['김고은', '안보현', '이유비', '박지현']",2021.0,0,세포들과 함께 먹고 사랑하고 성장하는 평범한 유미(김고은 분)의 이야기를 그린 세포...,0.0,https://image.tving.com/resize_cdn.php?u=https...,1.0,NaN
2,P001496087,환승연애,NaN,TV 프로그램,[],"['예능', '관찰리얼리티']","['이진주', '최미래', '이희선', '김슬기', '곽정은', '이승환', '신혜...","['사이먼도미닉', '이용진', '김예원', '유라']",2021.0,0,다양한 이유로 이별한 커플들이 모여 지나간 사랑을 되짚고 새로운 사랑을 찾아나가는 ...,0.0,https://image.tving.com/resize_cdn.php?u=https...,1.0,NaN
3,P000515167,놀라운 토요일,NaN,TV 프로그램,[],"['예능', '음악예능']","['이태경', '진보미', '강은비', '강유민', '김윤아', '임다영']","['신동엽', '김동현', '문세윤', '박나래', '넉살', '태연', '한해',...",2018.0,0,‘좋아요’ 폭발하는 SNS 핵 공감 핫 플레이스부터 오랜 전통이 살아 숨 쉬는 노포...,0.0,https://image.tving.com/resize_cdn.php?u=https...,1.0,NaN
4,P001515756,Show Me The Money 10,NaN,TV 프로그램,[],"['예능', '음악예능']",[],[],2021.0,0,Mnet <Show Me The Money 10>2021 COMING SOON,0.0,https://image.tving.com/resize_cdn.php?u=https...,1.0,NaN


In [153]:
len(tv_cou_df[tv_cou_df.duplicated(['title', 'category', 'director'])])

362

## tving, coupang 중복 데이터 제거

In [154]:
dup_index = tv_cou_df[tv_cou_df.duplicated(['title', 'category', 'director'], keep=False)].index

In [155]:
tv_cou_df[tv_cou_df['title'] == '걸캅스']

,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.tving,ott.coupang
2318,M000234633,걸캅스,Miss ＆ Mrs. Cops,영화,[],"['코미디', '']",['정다원'],"['라미란', '이성경', '윤상현', '최수영']",2018.0,0,′민원실 퇴출 0순위 전직 전설의 형사 ′미영′과 민원실로 밀려난 현직 꼴통 형사 ...,6.0,https://image.tving.com/upload/cms/caim/CAIM21...,1.0,NaN
7510,243d4144-f66e-4a68-af9a-e2c865f4585b,걸캅스,NaN,영화,[],"['코미디', '액션', '범죄']",['정다원'],"['라미란', '이성경']",NaN,107,민원실 퇴출 0순위 전직 전설의 형사 미영과 민원실로 밀려난 현직 꼴통 형사 지혜....,0.0,https://img.coupangstreaming.com/titles/243d41...,NaN,1.0


In [161]:
for idx in dup_index:
    if idx < 5753:
        # tving_index
        coupang_idx = search_dup_data_in_coupang(tv_cou_df.loc[idx]['title'], tv_cou_df)
        if coupang_idx is None:
            continue
        cur_data = tv_cou_df.loc[idx]
        for i in ['runtime', 'genre', 'actors', 'ott.coupang']:
            if len(str(cur_data[i])) <= len(str(tv_cou_df.loc[coupang_idx][i])):
                cur_data[i] = tv_cou_df.loc[coupang_idx][i]
        tv_cou_df = tv_cou_df.drop(coupang_idx)
        tv_cou_df.loc[idx] = cur_data

/usr/local/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [160]:
def search_dup_data_in_coupang(title, df):
    for idx in dup_index:
        if idx < 5753:
            continue
        # coupang_index
        try:
            com_title = df.loc[idx]['title']
        except:
            continue
        if df.loc[idx]['title'] == title:
            return idx

In [162]:
tv_cou_df[tv_cou_df['title'] == '걸캅스']

,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.tving,ott.coupang
2318,M000234633,걸캅스,Miss ＆ Mrs. Cops,영화,[],"['코미디', '액션', '범죄']",['정다원'],"['라미란', '이성경', '윤상현', '최수영']",2018.0,107,′민원실 퇴출 0순위 전직 전설의 형사 ′미영′과 민원실로 밀려난 현직 꼴통 형사 ...,6.0,https://image.tving.com/upload/cms/caim/CAIM21...,1.0,1.0


In [163]:
len(tv_cou_df[tv_cou_df.duplicated(['title', 'category', 'director'])])

0

In [164]:
len(tv_cou_df)

7182

# JustWatch + Tving Data + Coupang

In [165]:
total_data = pd.concat([jw_df, tv_cou_df], ignore_index=True)

In [166]:
len(total_data)

24937

In [167]:
total_data.head(2)

,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.coupang,ott.Netflix,ott.tving,ott.Watcha,ott.wavve
0,10,홈랜드,Homeland,TV 프로그램,['미국'],"['액션', '전쟁', '드라마', '스릴러', '범죄']",[],"['Claire Danes', 'Maury Sterling', 'Numan Acar...",2011.0,52.0,미 해병대 보병 부사관인 니콜라스 브로디 하사는 이라크 자유작전(이라크 전쟁) 중 ...,7.9,https://images.justwatch.com/poster/46536252/s332,None,['정액제'],None,None,None
1,112614,중경삼림,重慶森林,영화,['홍콩'],"['로맨스', '코미디', '범죄', '드라마']",['Wong Kar-wai'],"['Brigitte Lin', 'Tony Leung Chiu-wai', 'Faye ...",1994.0,103.0,"경찰 223은 헤어진 옛 애인을 기다리며, 1달 동안 그녀에게서 연락이 오지 않으면...",8.1,https://images.justwatch.com/poster/241789176/...,None,None,None,['정액제'],None


In [168]:
len(total_data[total_data.duplicated()])

0

# 중복 데이터 제거 ((Tving+Coupang)과 justwatch 겹치는 목록)

In [169]:
# 중복되는 인덱스의 배열
dup_index = total_data[total_data.duplicated(['title', 'category', 'released_year'], keep=False)].index
print(dup_index)

Int64Index([    3,     4,     6,     8,    10,    11,    12,    17,    21,
               24,
            ...
            23494, 23495, 23497, 23500, 23502, 23503, 23504, 23505, 23506,
            23507],
           dtype='int64', length=4234)


In [174]:
total_data[total_data['title'] == '유주얼 서스펙트']

,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.coupang,ott.Netflix,ott.tving,ott.Watcha,ott.wavve
3,7320,유주얼 서스펙트,The Usual Suspects,영화,['미국'],"['스릴러', '드라마', '범죄']",['Bryan Singer'],"['Gabriel Byrne', 'Stephen Baldwin', 'Kevin Sp...",1995.0,106.0,"산 페드로 부두에서 27명이 사망하고 9,100만달러가 증발하는 유혈극이 벌어진다....",8.3,https://images.justwatch.com/poster/8655298/s332,None,['정액제'],None,None,"['정액제', '구매', '대여']"
20942,M000321034,유주얼 서스펙트,The Usual Suspects,영화,[],"['스릴러', '']",['브라이언 싱어'],"['스티븐 볼드윈', '가브리엘 번', '베네치오 델 토로', '채즈 팔민테리', ...",1995.0,0.0,"산페드로 부두, 27명이 사망하고 9,100만달러가 증발하는 유혈극이 벌어진다. 수...",9.0,https://image.tving.com/upload/cms/caim/CAIM21...,NaN,NaN,1.0,NaN,NaN


In [177]:
total_data[(total_data['title']=='괴물') & (total_data['category']=='영화') & (total_data['released_year']==2006)]

,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.coupang,ott.Netflix,ott.tving,ott.Watcha,ott.wavve
2931,28886,괴물,괴물,영화,['대한민국'],"['공포', '애니메이션', 'SF', '액션', '드라마']",['Bong Joon-ho'],"['Song Kang-ho', 'Byun Hee-bong', 'Park Hae-il...",2006.0,120.0,"아버지가 운영하는 한강매점, 늘어지게 낮잠 자던 강두는 우연히 특이한 광경을 목격하...",7.0,https://images.justwatch.com/poster/193809351/...,None,['정액제'],None,['정액제'],"['구매', '대여']"
21957,M000292336,괴물,The Host,영화,[],"[' 드라마', '패밀리', '액션', 'SF']",['봉준호'],"[' 송강호', ' 변희봉', '박해일', '배두나', '고아성']",2006.0,119.0,"햇살 가득한 평화로운 한강 둔치 아버지가 운영하는 한강 매점, 늘어지게 낮잠 자던 ...",9.0,https://image.tving.com/upload/cms/caim/CAIM21...,1.0,NaN,1.0,NaN,NaN


In [171]:
len(total_data)

24937

In [180]:
for idx in dup_index:
    if idx < 17764:
        # justwatch data
        tving_idx = search_dup_data_in_tving(total_data.loc[idx])
        if tving_idx is None:
            print("error")
            continue
        cur_data = total_data.loc[idx]
        for i in ['director', 'actors', 'ott.tving', 'ott.coupang']:
            cur_data[i] = total_data.loc[tving_idx][i]
        total_data = total_data.drop(tving_idx)
        total_data.loc[idx] = cur_data

error
error
error
error
error
error
error
error
error
error
error


In [178]:
def search_dup_data_in_tving(data):
    for idx in dup_index:
        if idx < 17764:
            continue
        try:
            com_data = total_data.loc[idx]
        except:
            continue
        if com_data['title'] == data['title'] and com_data['category'] == data['category'] and com_data['released_year'] == data['released_year']:
            return idx

In [181]:
len(total_data)

22831

In [182]:
total_data[total_data['title']=='아수라']

,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.coupang,ott.Netflix,ott.tving,ott.Watcha,ott.wavve
5477,64123,아수라,アシュラ,영화,['일본'],"['공포', '애니메이션', '액션', '드라마', '역사']",['Keiichi Sato'],"['Masako Nozawa', 'Hiroaki Hirata', 'Kinya Kit...",2012.0,76.0,“태어나지 않는 편이 좋았을 텐데…” 15세기 중기(무로마치 시대)의 교토는 그야...,6.8,https://images.justwatch.com/poster/94291387/s332,None,None,None,None,"['정액제', '대여']"
8593,247629,아수라,아수라,영화,['대한민국'],"['범죄', '액션', '드라마', '스릴러']",['김성수'],"['정우성', '황정민', '주지훈', '곽도원']",2016.0,136.0,강력계 형사 한도경은 이권과 성공을 위해 각종 범죄를 저지르는 악덕시장 박성배의 뒷...,6.7,https://images.justwatch.com/poster/9627223/s332,1.0,['정액제'],1.0,None,"['구매', '대여']"
22405,M000144672,아수라,Asura,영화,[],"['공포(호러)', '']",['사토 케이이치'],"['노자와 마사코', '하야시바라 메구미']",2013.0,0.0,“태어나지 않는 편이 좋았을 텐데...”15세기 중기(무로마치 시대)의 교토는 그야...,9.0,https://image.tving.com/upload/cms/caim/CAIM21...,NaN,NaN,1.0,NaN,NaN


# 중복 데이터 제거 2 (노가다)

In [193]:
from collections import defaultdict
title_category = defaultdict(list)
for i in total_data[total_data.duplicated(['title', 'category'], keep=False)].index:
    cur = total_data.loc[i]
    title_category[(cur['title'], cur['category'])].append(i)

In [196]:
dict(filter(lambda x: len(x[1]) > 2, title_category.items()))

{('비트', '영화'): [89, 14140, 24152],
 ('로미오와 줄리엣', '영화'): [157, 297, 7001],
 ('나쁜 녀석들', '영화'): [175, 6718, 24477],
 ('차이나타운', '영화'): [254, 7489, 21218],
 ('인페르노', '영화'): [322, 8642, 24318],
 ('벤허', '영화'): [368, 853, 8803, 24565],
 ('뮬란', '영화'): [376, 15401, 21960],
 ('엠마', '영화'): [400, 15411, 24589],
 ('순수의 시대', '영화'): [420, 7827, 24209],
 ('하울링', '영화'): [465, 5220, 23058],
 ('위대한 유산', '영화'): [483, 582, 2378, 5874],
 ('적과의 동침', '영화'): [574, 4986, 17042, 21991],
 ('은밀한 유혹', '영화'): [583, 8075, 20604],
 ('천녀유혼', '영화'): [620, 4919, 21547],
 ('스트리트 파이터', '영화'): [680, 1516, 24629],
 ('젊은이의 양지', '영화'): [714, 15433, 21137],
 ('연인', '영화'): [737, 2525, 24227],
 ('안나 카레니나', '영화'): [748, 1414, 5172],
 ('열혈남아', '영화'): [854, 3018, 24343],
 ('리벤지', '영화'): [894, 11697, 20730],
 ('레 미제라블', '영화'): [928, 13661, 20488],
 ('소공녀', '영화'): [1251, 11672, 24377],
 ('모모', '영화'): [1293, 4127, 9214],
 ('빈센트', '영화'): [1315, 7073, 10674],
 ('기문둔갑', '영화'): [1669, 10576, 15843],
 ('암전', '영화'): [1704, 14304, 20365],
 ('장

In [376]:
total_data[total_data['title'] == '로미오와 줄리엣']

,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.coupang,ott.Netflix,ott.tving,ott.Watcha,ott.wavve
157,109186,로미오와 줄리엣,Romeo + Juliet,영화,['미국'],"['드라마', '로맨스', 'Made in Europe']",['Baz Luhrmann'],"['Leonardo DiCaprio', 'Claire Danes', 'Jesse B...",1996.0,120.0,"정열의 도시 베로나, 태양의 열기가 가득한 이 도시에서 두 가문 몬태규가와 캐플릿가...",6.8,https://images.justwatch.com/poster/15554797/s332,None,None,None,None,['구매']
297,18906,로미오와 줄리엣,Romeo and Juliet,영화,"['이탈리아', '영국']","['로맨스', 'Made in Europe', '드라마']",['Franco Zeffirelli'],"['Leonard Whiting', 'Olivia Hussey', 'John McE...",1968.0,138.0,몬태규가의 로미오는 원수 집안인 캐플릿가의 가면파티에 몰래 갔다 우연히 아름다운 여...,7.5,https://images.justwatch.com/poster/16035808/s332,None,['정액제'],None,None,"['구매', '대여']"
7001,147996,로미오와 줄리엣,Romeo and Juliet,영화,['미국'],"['로맨스', '드라마', '코미디', '판타지', 'SF', 'Made in Eu...",['David Leveaux'],"['Orlando Bloom', 'Condola Rashad', 'Christian...",2014.0,135.0,Orlando Bloom and Condola Rashad take on the t...,6.0,https://images.justwatch.com/poster/10252806/s332,None,None,None,['정액제'],None


In [372]:
total_data = tmp_func(8642, 24318, total_data)

In [368]:
def tmp_func(cur_idx, other_idx, total_data):
    cur, other = total_data.loc[cur_idx], total_data.loc[other_idx]
    for i in ['released_year', 'ott.tving', 'ott.coupang', 'director', 'actors']:
        if i == 'ott.tving' and not pd.isna(cur[i]):
            continue
        cur[i] = other[i]
    total_data.loc[cur_idx] = cur
    total_data = total_data.drop(other_idx)
    return total_data

In [192]:
for key, val in title_category.items():
    val.sort()
    if len(val) == 2 and (val[0] < 17764 and val[1] >= 17764):
        cur = total_data.loc[val[0]]
        other = total_data.loc[val[1]]
        for i in ['ott.tving', 'ott.coupang', 'director', 'actors']:
            cur[i] = other[i]
        total_data.loc[val[0]] = cur
        total_data = total_data.drop(val[1])

/usr/local/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


# Country의 결측치 => 삭제?

In [377]:
len(total_data[total_data['country'] == '[]'])

4906

# Actor, Director의 결측치와 영어데이터 => 삭제? or검색을 활용한 방법??

In [378]:
len(total_data[total_data['actors'] == '[]'])

1907

In [379]:
import string
eng_actor_cnt = 0
for i in total_data['actors']:
    if len(i) > 3 and i[2] in string.ascii_letters:
        eng_actor_cnt += 1
eng_actor_cnt

13599

In [380]:
eng_dir_cnt = 0
for i in total_data['director']:
    if len(i) > 3 and i[2] in string.ascii_letters:
        eng_dir_cnt += 1
eng_dir_cnt

11673

In [381]:
len(total_data[total_data['director'] == '[]'])

3851

In [382]:
mask1 = (total_data['actors'] == '[]') & (total_data['director'] == '[]')

In [383]:
len(total_data.loc[mask1, :])

1120

# 장르로 거르는건?

In [384]:
len(total_data[total_data['genre'] == '[]'])

689

In [385]:
tmp = set()
for i in total_data['genre'].unique():
    for j in i[1:-1].split(","):
        tmp.add(j)

In [386]:
print(len(tmp))

222


# 영어 줄거리는 과감하게 버리는 건?

In [393]:
import re
reg = re.compile('[ㄱ-힣]')
cnt = 0 
for i in total_data.index:
    cur = total_data.loc[i]
    try:
        if reg.search(cur['summary']):
            pass
        else:
            cnt+=1
            pass
    except:
        pass
cnt

0

# 영어 데이터 처리

In [390]:
import re
reg = re.compile('[ㄱ-힣]')
cnt = 0 
for i in total_data.index:
    cur = total_data.loc[i]
    try:
        if reg.search(cur['summary']):
            pass
        else:
            cur['summary'] = None
            pass
    except:
        cur['summary'] = None
    total_data.loc[i] = cur

In [391]:
total_data[total_data['title']=='아기상어: 올리와 윌리엄']

,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.coupang,ott.Netflix,ott.tving,ott.Watcha,ott.wavve
17745,269837,아기상어: 올리와 윌리엄,아기상어: 올리와 윌리엄,TV 프로그램,['대한민국'],[],[],"['Kimiko Glenn', 'Luke Youngblood', 'Natasha R...",2021.0,11.0,None,0.0,https://images.justwatch.com/poster/239863093/...,None,None,None,['정액제'],['정액제']


In [394]:
len(total_data)

21778

In [395]:
total_data.isnull().sum()

id                    0
title                 0
original_title     2728
category              0
country               0
genre                 0
director              0
actors                0
released_year       941
runtime              22
summary            1845
rating                0
imgurl              218
ott.coupang       19988
ott.Netflix       16755
ott.tving         16114
ott.Watcha        14648
ott.wavve          9064
dtype: int64

In [399]:
total_data.to_csv("/Users/jeonggyu/Desktop/수집한 dataset/new_data/total_data.csv")

# actor 이름 교체

In [400]:
df = pd.read_csv("/Users/jeonggyu/Desktop/수집한 dataset/new_data/total_data.csv")

In [402]:
with open("/Users/jeonggyu/Desktop/수집한 dataset/actors/final_actors.json", "r") as f:
    dict_actor = json.load(f)
    for idx in df.index:
        cur = df.loc[idx]
        try:
            cur_actors = cur['actors'].replace("'", "")[1:-1].split(",")

            ch_actors = list(map(lambda x: x.strip(), filter(lambda x: x is not None, map(lambda x: dict_actor.get(x), cur_actors))))
            cur['actors'] = str(ch_actors)

            df.loc[idx] = cur
        except Exception as e:
            pass
#             print(e)

/usr/local/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [405]:
df.head(5)

,Unnamed: 0,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.coupang,ott.Netflix,ott.tving,ott.Watcha,ott.wavve
0,0,10,홈랜드,Homeland,TV 프로그램,['미국'],"['액션', '전쟁', '드라마', '스릴러', '범죄']",[],"['클레어 데인즈', '모리 스터링', '너맨 아카', '님랏 카우르', '맨디 파...",2011.0,52.0,미 해병대 보병 부사관인 니콜라스 브로디 하사는 이라크 자유작전(이라크 전쟁) 중 ...,7.9,https://images.justwatch.com/poster/46536252/s332,NaN,['정액제'],NaN,NaN,NaN
1,1,112614,중경삼림,重慶森林,영화,['홍콩'],"['로맨스', '코미디', '범죄', '드라마']",['왕가위'],"['임청하', '양조위', '왕페이', '금성무', '주가령', '청 싱 초', '...",1994.0,103.0,"경찰 223은 헤어진 옛 애인을 기다리며, 1달 동안 그녀에게서 연락이 오지 않으면...",8.1,https://images.justwatch.com/poster/241789176/...,NaN,NaN,NaN,['정액제'],NaN
2,2,128565,캔디맨,Candyman,영화,['미국'],"['드라마', '공포', '스릴러']",['버나드 로즈'],"['버지니아 매드슨', '토니 토드', '잰더 버클리', '카시 레몬즈', '바네사...",1992.0,99.0,도시의 전설에 대한 논문을 쓰고 있는 헬렌 라일(Helen Lyle: 버지니아 매드...,6.6,https://images.justwatch.com/poster/10703948/s332,NaN,NaN,NaN,NaN,"['정액제', '대여']"
3,3,7320,유주얼 서스펙트,The Usual Suspects,영화,['미국'],"['스릴러', '드라마', '범죄']",['브라이언 싱어'],"['스티븐 볼드윈', '가브리엘 번', '베네치오 델 토로', '채즈 팔민테리', ...",1995.0,106.0,"산 페드로 부두에서 27명이 사망하고 9,100만달러가 증발하는 유혈극이 벌어진다....",8.3,https://images.justwatch.com/poster/8655298/s332,NaN,['정액제'],1.0,NaN,"['정액제', '구매', '대여']"
4,4,42409,파이트 클럽,Fight Club,영화,"['독일', '미국']",['드라마'],['데이빗 핀처'],"['브래드 피트', '에드워드 노튼', '헬레나 본햄 카터', '미트 로프', '자...",1999.0,139.0,자동차 회사의 리콜 심사관으로 일하는 주인공(에드워드 노튼)은 일상의 무료함과 공허...,8.6,https://images.justwatch.com/poster/21851876/s332,1.0,['정액제'],1.0,NaN,['구매']


# director 이름 교체

In [404]:
with open("/Users/jeonggyu/Desktop/수집한 dataset/director/final_director.json") as f:
    dict_dir = json.load(f)
    for idx in df.index:
        cur = df.loc[idx]
        cur_dir = cur['director'].replace("'", "")[1:-1].split(",")
        
        ch_dir = list(map(lambda x: x.strip(), filter(lambda x: x is not None, map(lambda x: dict_dir.get(x), cur_dir))))
        
        cur['director'] = str(ch_dir)
        df.loc[idx] = cur

/usr/local/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [296]:
with open(f"/Users/jeonggyu/Desktop/수집한 dataset/director/director.json", "r") as nf,\
     open(f"/Users/jeonggyu/Desktop/수집한 dataset/director/final_director.json", "w") as target:
    nd = json.load(nf)
    print(len(nd.keys()))

    # 한글 key 값들 처리
    for key, val in nd.items():
        if reg.search(key):
            nd[key] = key
    
    
    tmp = dict(filter(lambda x: reg.search(str(x[1])), nd.items()))
    
    print(len(tmp.keys()))
    target.write(json.dumps(tmp))

13959
11339


In [13]:
df.to_csv("/Users/jeonggyu/Desktop/수집한 dataset/data/final_total_data.csv")

# actor 데이터 합치기

In [360]:
import re
reg = re.compile('[가-힣]')

file_name = [('0', '7214'), ('7214', '25000'), ('25000', '50000'), ('50000', '75000'), ['75000', '100000'], ['100000', 'end']]
for start, end in file_name:
    with open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/google_{start}_{end}.json', "r") as nf,\
         open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/{start}_{end}.json', "r") as of,\
         open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/final_{start}_{end}.json', "w") as target:
        nd, od = json.load(nf), json.load(of)
        print("naver = ", len(od), len(list(filter(lambda x: x[1] is None, od.items()))))
        print("google = ", len(nd), len(list(filter(lambda x: x[1] is None, nd.items()))))
        # google검색을 naver와 합친다.
        od.update(nd)

        # 한글 key 값들 처리
        for key, val in od.items():
            if reg.search(key):
                od[key] = key

        # none 처리
        tmp = dict(filter(lambda x: x[1] is not None, od.items()))

        # 영어 value 값 처리
        tmp = dict(filter(lambda x: reg.search(str(x[1])), tmp.items()))

        print("naver + google = ", len(tmp))

        target.write(json.dumps(tmp))


naver =  7214 4065
google =  4065 1269
naver + google =  5852
naver =  17786 11851
google =  11851 4405
naver + google =  13004
naver =  25000 15191
google =  15191 5867
naver + google =  18702
naver =  25000 17172
google =  17172 6908
naver + google =  17486
naver =  25000 19026
google =  19026 7901
naver + google =  16296
naver =  37411 27341
google =  27341 12744
naver + google =  24737


In [362]:
with open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/final_0_7214.json', "r") as f1,\
     open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/final_7214_25000.json', "r") as f2,\
     open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/final_25000_50000.json', "r") as f3,\
     open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/final_50000_75000.json', "r") as f4, \
     open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/final_75000_100000.json', "r") as f5, \
     open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/final_100000_end.json', "r") as f6, \
     open(f'/Users/jeonggyu/Desktop/수집한 dataset/actors/final_actors.json', "w") as target:
    d1, d2, d3, d4, d5, d6 = json.load(f1), json.load(f2), json.load(f3), json.load(f4), json.load(f5), json.load(f6)
    print(len(d1), len(d2), len(d3), len(d4), len(d5), len(d6))
    d1.update(d2)
    d1.update(d3)
    d1.update(d4)
    d1.update(d5)
    d1.update(d6)
    print(len(d1))
    target.write(json.dumps(d1))
    

5852 13004 18702 17486 16296 24737
96077


In [448]:
df = df.drop(columns=['Unnamed: 0', 'original_title', 'country'])

In [450]:
js = df.to_json("/Users/jeonggyu/Desktop/수집한 dataset/data/final_total_data.json", orient="index")

In [437]:
len(df['director'].unique())
    

9117

In [447]:
abcd = set()
for i in df['actorss'].unique():
    for val in ast.literal_eval(i):
        abcd.add(val)s
len(abcd)

83169

In [3]:
import pandas as pd

In [6]:
import csv
import json

input_file_name = "/Users/jeonggyu/Desktop/수집한 dataset/data/final_total_data.json"
output_file_name = "/Users/jeonggyu/Desktop/수집한 dataset/data/final_total_data.csv"

with open(input_file_name, "r", encoding="utf-8", newline="") as input_file, \
        open(output_file_name, "w", encoding="utf-8", newline="") as output_file:

    data = []
    for line in input_file:
        datum = json.loads(line)
        data.append(datum)
        
    csvwriter = csv.writer(output_file)
    csvwriter.writerow(data[0].keys())
    for line in data:
        csvwriter.writerow(line.values())

In [103]:
df = pd.read_csv("/Users/jeonggyu/Desktop/수집한 dataset/data/final_total_data.csv")

In [104]:
len(df)

21771

In [105]:
df = df.drop(columns=["Unnamed: 0", "Unnamed: 0.1", "original_title", "country"])

In [106]:
df.head(5)

,title,category,genre,director,actors,released_year,runtime,summary,rating,ott.Netflix,ott.Watcha,ott.wavve,ott.tving,ott.coupang
0,홈랜드,TV 프로그램,"['전쟁', '드라마', '액션', '범죄', '스릴러']",[],"['클레어 데인즈', '모리 스터링', '너맨 아카', '님랏 카우르', '맨디 파...",2011.0,52.0,미 해병대 보병 부사관인 니콜라스 브로디 하사는 이라크 자유작전(이라크 전쟁) 중 ...,7.9,['정액제'],NaN,NaN,NaN,NaN
1,중경삼림,영화,"['로맨스', '드라마', '코미디', '범죄']",['왕가위'],"['임청하', '양조위', '왕페이', '금성무', '주가령', '청 싱 초', '...",1994.0,103.0,"경찰 223은 헤어진 옛 애인을 기다리며, 1달 동안 그녀에게서 연락이 오지 않으면...",8.1,NaN,['정액제'],NaN,NaN,NaN
2,캔디맨,영화,"['드라마', '공포', '스릴러']",['버나드 로즈'],"['버지니아 매드슨', '토니 토드', '잰더 버클리', '카시 레몬즈', '바네사...",1992.0,99.0,도시의 전설에 대한 논문을 쓰고 있는 헬렌 라일(Helen Lyle: 버지니아 매드...,6.6,NaN,NaN,"['정액제', '대여']",NaN,NaN
3,파이트 클럽,영화,['드라마'],['데이빗 핀처'],"['브래드 피트', '에드워드 노튼', '헬레나 본햄 카터', '미트 로프', '자...",1999.0,139.0,자동차 회사의 리콜 심사관으로 일하는 주인공(에드워드 노튼)은 일상의 무료함과 공허...,8.6,['정액제'],NaN,['구매'],['구매'],['정액제']
4,유주얼 서스펙트,영화,"['스릴러', '드라마', '범죄']",['브라이언 싱어'],"['스티븐 볼드윈', '가브리엘 번', '베네치오 델 토로', '채즈 팔민테리', ...",1995.0,106.0,"산 페드로 부두에서 27명이 사망하고 9,100만달러가 증발하는 유혈극이 벌어진다....",8.3,['정액제'],NaN,"['정액제', '대여', '구매']",['구매'],NaN


In [114]:
import ast
a = set()
for i in df['genre'].unique():
    for val in ast.literal_eval(i):
        a.add(val)

In [115]:
a

{'SF',
 'e스포츠 리그',
 '가족',
 '가족 예능',
 '강연',
 '갱스터',
 '건강',
 '게임',
 '게임 버라이어티',
 '게임쇼',
 '고전 키즈',
 '공룡',
 '공연',
 '공연실황',
 '공포',
 '과학',
 '관찰리얼리티',
 '교양',
 '교육',
 '국제대회',
 '권력',
 '기술',
 '나영석 예능',
 '낚시',
 '노래',
 '느와르',
 '다큐멘터리',
 '다크 코미디',
 '단막극',
 '당구',
 '독서 예능',
 '동물',
 '동요',
 '동화',
 '드라마',
 '디지털오리지널',
 '로맨스',
 '로맨틱 코미디',
 '로봇',
 '리빙',
 '리얼리티',
 '마법',
 '마법과 모험',
 '막장',
 '먹방',
 '메디컬',
 '멜로',
 '명랑',
 '모험',
 '뮤지컬',
 '뮤직',
 '미국',
 '미스터리',
 '밀리터리',
 '버라이어티',
 '범죄',
 '범죄 스릴러',
 '법정',
 '복수',
 '뷰티',
 '비하인드',
 '사극',
 '생활꿀팁',
 '서바이벌',
 '서부',
 '서부극',
 '선발대회',
 '성인',
 '성장',
 '소설 원작',
 '수사극',
 '수사물',
 '순정',
 '스릴러',
 '스타일',
 '스페셜',
 '스포츠',
 '스포츠 예능',
 '시대극',
 '시리즈물',
 '시사',
 '시상식',
 '시트콤',
 '실화',
 '실화 바탕',
 '아이돌',
 '애니',
 '애니멀',
 '애니메이션',
 '액션',
 '어드벤처',
 '어린이',
 '에듀테인먼트',
 '여성주인공',
 '여행',
 '역사',
 '연애',
 '연애 리얼리티',
 '엽기',
 '영국',
 '영어교육',
 '영유아',
 '영화',
 '예능',
 '예능 스페셜',
 '오컬트',
 '오피스 드라마',
 '요리',
 '요리 서바이벌',
 '워맨스',
 '웹드라마',
 '웹예능',
 '웹툰',
 '유아',
 '육아',
 '음식',
 '음악',
 '음악 서바이벌',
 '음악예능',
 '의학',
 '인포테인먼

# 뺄 것들
- ''
- '3d'
- 'Made in Europe'
- 'Reality TV'
- 'sf' => 'SF'
- '건강/의학' => '건강', '의학'
- '공포(호러)' => '공포'
- '공포/스릴러' => '공포', '스릴러'
- '공포/오컬트' => '공포', '오컬트'
- '교육&독서 예능' => '교육', '독서 예능'
- '노래/동요' => '노래', '동요'
- '동물 친구들' => '동물'
- '로맨스/순정' => '로맨스', '순정'
- '로봇/액션' => '로봇', '액션'
- '리빙/재테크' => '리빙', '재테크'
- '멜로/로맨스' => '멜로', '로맨스'
- '모험/판타지' => '모험', '판타지'
- '미스터리/스릴러' => '미스터리', '스릴러'
- '서부극(웨스턴) => '서부극'
- '스페셜 몰아보기' => '스페셜'
- '시사・교양' => '시사', '교양'
- '실화 바탕' => '실화'
- '어드밴처', '어드벤처', '어드벤쳐' => '어드벤쳐'
- '연애 리얼리티&토크쇼' => '연애 리얼리티', '토크쇼'
- '자동차,기차, 탈 것' => 없애자
- '자동차와 비행기' => 없애자
- '청춘(성장)' => '청춘'
- '코메디' => '코미디'
- '토크쇼' => '토크'

In [406]:
change_dict = {"" : "del","3d" : "del", "Made in Europe": "del", "Reality TV": "del", "sf": "SF", "공포(호러)": "공포", "동물 친구들": "동물", "서부극(웨스턴)": "서부극", "스페셜 몰아보기" : "스페셜", "어드밴처" : "어드벤처", "어드벤쳐" : "어드벤처", "자동차,기차, 탈 것" : "del", "자동차와 비행기" : "del", "청춘(성장)": "청춘", "코메디" : "코미디", "토크쇼" : "토크", "아이돌(리얼리티)": "아이돌"}

In [407]:
def split_func(val_list):
    ans = []
    for val in val_list:
        for v in val.replace("&", "/").replace("・", "/").split("/"):
            ans.append(v.strip())
    return ans

In [409]:
import ast
def func(x):
    
    if change_dict.get(x) is None:
        return x
    elif change_dict.get(x) == "del":
        return
    return change_dict.get(x)
    
for idx in df.index:
    cur = df.loc[idx]
    try:
        # 먼저 "/", "&"를 나누는 것부터 하자.
        cur_genres = split_func(ast.literal_eval(cur['genre']))

        ch_genres = list(filter(lambda x: x is not None, map(func, cur_genres)))

        cur['genre'] = str(list(set(ch_genres)))

        df.loc[idx] = cur
    except Exception as e:
        pass
        print(e)

/usr/local/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [410]:
df.head(5)

,Unnamed: 0,id,title,original_title,category,country,genre,director,actors,released_year,runtime,summary,rating,imgurl,ott.coupang,ott.Netflix,ott.tving,ott.Watcha,ott.wavve
0,0,10,홈랜드,Homeland,TV 프로그램,['미국'],"['범죄', '액션', '드라마', '전쟁', '스릴러']",[],"['클레어 데인즈', '모리 스터링', '너맨 아카', '님랏 카우르', '맨디 파...",2011.0,52.0,미 해병대 보병 부사관인 니콜라스 브로디 하사는 이라크 자유작전(이라크 전쟁) 중 ...,7.9,https://images.justwatch.com/poster/46536252/s332,NaN,['정액제'],NaN,NaN,NaN
1,1,112614,중경삼림,重慶森林,영화,['홍콩'],"['로맨스', '코미디', '범죄', '드라마']",['왕가위'],"['임청하', '양조위', '왕페이', '금성무', '주가령', '청 싱 초', '...",1994.0,103.0,"경찰 223은 헤어진 옛 애인을 기다리며, 1달 동안 그녀에게서 연락이 오지 않으면...",8.1,https://images.justwatch.com/poster/241789176/...,NaN,NaN,NaN,['정액제'],NaN
2,2,128565,캔디맨,Candyman,영화,['미국'],"['스릴러', '공포', '드라마']",['버나드 로즈'],"['버지니아 매드슨', '토니 토드', '잰더 버클리', '카시 레몬즈', '바네사...",1992.0,99.0,도시의 전설에 대한 논문을 쓰고 있는 헬렌 라일(Helen Lyle: 버지니아 매드...,6.6,https://images.justwatch.com/poster/10703948/s332,NaN,NaN,NaN,NaN,"['정액제', '대여']"
3,3,7320,유주얼 서스펙트,The Usual Suspects,영화,['미국'],"['범죄', '드라마', '스릴러']",['브라이언 싱어'],"['스티븐 볼드윈', '가브리엘 번', '베네치오 델 토로', '채즈 팔민테리', ...",1995.0,106.0,"산 페드로 부두에서 27명이 사망하고 9,100만달러가 증발하는 유혈극이 벌어진다....",8.3,https://images.justwatch.com/poster/8655298/s332,NaN,['정액제'],1.0,NaN,"['정액제', '구매', '대여']"
4,4,42409,파이트 클럽,Fight Club,영화,"['독일', '미국']",['드라마'],['데이빗 핀처'],"['브래드 피트', '에드워드 노튼', '헬레나 본햄 카터', '미트 로프', '자...",1999.0,139.0,자동차 회사의 리콜 심사관으로 일하는 주인공(에드워드 노튼)은 일상의 무료함과 공허...,8.6,https://images.justwatch.com/poster/21851876/s332,1.0,['정액제'],1.0,NaN,['구매']


In [411]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [412]:
df.to_csv("/Users/jeonggyu/Desktop/수집한 dataset/new_data/final_total_data.csv")
df.to_json("/Users/jeonggyu/Desktop/수집한 dataset/new_data/final_total_data.json", orient="index")